In [ ]:
import os
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from huggingface_hub import login
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline

In [ ]:
login(token=os.environ['HF_TOKEN'])

In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
es_client = Elasticsearch('http://localhost:9200')
index_name = "homework-questions"

In [ ]:
def es_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
QUESTION: {question}

CONTEXT:
{context}

ANSWER: 
""".strip()
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question = doc['question'], text = doc['text']) + "\n\n"
        
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
def llm(prompt):
    response = generator(prompt, max_length=500, num_return_sequences=1, truncation=True)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()

In [ ]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [ ]:
query = 'may I still enroll if the course has already started?'

In [ ]:
rag(query)
#llm(query)